In [1]:
import plotly.express as px
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

#import csv
pd.set_option('display.max_rows', None)  # Display all rows

In [2]:
day = "10"
month = "10"
dataset = "ble"

date = day + "_" + month #change date accordingly
date_year = "2023-" + month + "-" + day
if date == "11_10":
    base = 1
else:
    base = 0

if base:
    folder_path = dataset + '_base/'
else:
    folder_path = dataset +'_'+ date + '/'

def process_file(file_number, date):
    file_path = os.path.join(folder_path, f'{dataset}_{date}_{file_number}.csv')

    df = pd.read_csv(file_path, parse_dates=['ts'])

    # Remove first empty column if it exists
    if df.shape[1] == 4:
        df = df.drop(df.columns[0], axis=1)
    # Remove entries with 'gabarit' equal to 1 or 2
    df_filt = df[(df['id'] == 2) | (df['id'] == 1)]

    # Write the updated DataFrame back to the CSV file
    #df.to_csv(file_path, index = False)
    # Reset the index of the DataFrame
    df_filt = df_filt.reset_index(drop = True)

    return df_filt

df1 = process_file(1,date)
df2 = process_file(2,date)
df3 = process_file(3,date)


In [3]:
def plot_ble(df,station):
    # Plotly line graph
    fig = px.line(df, x='ts', y='rssi', color='id', title=f'Line Graph for Station {station}')
    fig.update_layout(xaxis_title='Timestamp', yaxis_title='RSSI', legend_title='ID')
    fig.show()

plot_ble(df1,station =1)
plot_ble(df2, station =2)
plot_ble(df3, station = 3)

#1 fim 18:02:00 -> 18:06:00
#2 inicio 17:57:30 -> 18:01:00

#3 monta cargas 18:07:00 -> 18:10:00

c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()


c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



c:\Users\isacc\miniconda3\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [4]:
# FIND BASE VALUES
def find_ref_ble_1_2(df1,df2,df3, start_time, end_time):

    ref_values = np.zeros((3,3))
    #############################################################
    #                              station 1 station 2 station 3
    #beacon placed in station 1     X         X         X
    #beacon placed in station 2     X         X         X
    #beacon placed in station 3     X         X         X

    for beacon in [1,2]:
        df1_id = df1[df1['id'] == beacon]
        df2_id = df2[df2['id'] == beacon]
        df3_id = df3[df3['id'] == beacon]

        

        # Filter DataFrame based on the time range
        df1_1 = df1_id[(df1_id['ts'] >= start_time) & (df1_id['ts'] <= end_time)]
        #av_{station}_{time_interval}
        med1_1 = df1_1['rssi'].median()
        av1_1 = df1_1['rssi'].mean()

        df2_1 = df2_id[(df2_id['ts'] >= start_time) & (df2_id['ts'] <= end_time)]
        med2_1 = df2_1['rssi'].median()
        av2_1 = df2_1['rssi'].mean()

        df3_1 = df3_id[(df3_id['ts'] >= start_time) & (df3_id['ts'] <= end_time)]
        med3_1 = df3_1['rssi'].median()
        av_3_1 = df3_1['rssi'].mean()

        #print(f'beacon: {beacon}')
        #beacon_ref_med = {'receiver1': med1_1, 'receiver2': med2_1, 'receiver3': med3_1}
        #beacon_ref_av = {'receiver1': av1_1, 'receiver2': av2_1, 'receiver3': av_3_1}
        ref_values[beacon-1,:] = [av1_1,av2_1,av_3_1]
        #print(ref_values)

    return ref_values

def find_ref_3(df1,df2,df3, ref_values,start_time, end_time):
    beacon = 2 #placed in station 3
    df1_id = df1[df1['id'] == beacon]
    df2_id = df2[df2['id'] == beacon]
    df3_id = df3[df3['id'] == beacon]
    # Filter DataFrame based on the time range
    df1_1 = df1_id[(df1_id['ts'] >= start_time) & (df1_id['ts'] <= end_time)]
    med1_1 = df1_1['rssi'].median()
    av1_1 = df1_1['rssi'].mean()

    df2_1 = df2_id[(df2_id['ts'] >= start_time) & (df2_id['ts'] <= end_time)]
    med2_1 = df2_1['rssi'].median()
    av2_1 = df2_1['rssi'].mean()

    df3_1 = df3_id[(df3_id['ts'] >= start_time) & (df3_id['ts'] <= end_time)]
    med3_1 = df3_1['rssi'].median()
    av_3_1 = df3_1['rssi'].mean()

    ref_values[2,:] = [av1_1,av2_1,av_3_1]
    ref_values[2,:] = [-60, -70, -50]

    return ref_values

if base == 1:
    start_time = '2023-10-11 19:00:00'
    end_time = '2023-10-12 07:00:00'
    ref_values = find_ref_ble_1_2(df1,df2,df3, start_time, end_time)

    #start_time = '2023-10-13 19:00:00'
    #end_time = '2023-10-14 07:00:00'
    ref_values = find_ref_3(df1,df2,df3, ref_values,start_time,end_time)
    print(ref_values)
else:
    ref_values = np.array([[-44.23451777, -80,         -66.21052632],
    [-85.8369906,  -66.46572438, -87.02805611],
    [-60,         -70,         -50,        ]])


In [5]:
def analyse_df(df1,df2,df3, beacon):
    # Filter DataFrames for the given id value
    #display(df1,df2,df3)
    #CHECK
    df1_id = df1[(df1['id'] == beacon) & (df1['rssi'] != -100)].reset_index(drop = True)
    df2_id = df2[(df2['id'] == beacon) & (df2['rssi'] != -100)].reset_index(drop = True)
    df3_id = df3[(df3['id'] == beacon) & (df3['rssi'] != -100)].reset_index(drop = True)
    
    id_max = df1_id.shape[0]
    df1_id = df1_id.loc[:id_max]
    df2_id = df2_id.loc[:id_max]
    df3_id = df3_id.loc[:id_max]
    #display(df1_id,df2_id,df3_id)
    df_rssi = pd.DataFrame(columns = ['ts','rssi1','rssi2','rssi3','station','x','y'])
    id = 0

    # Iterate through timestamps in df1
    #display(df1_id, df2_id, df3_id)
    for index, row in df1_id.iterrows():
        # Find the current timestamp in df1
        #print("ts")
        #ts_current = df1_id.loc[index, 'ts']
        ts_current = pd.to_datetime(row['ts'])
        df_rssi.loc[id, 'ts'] = ts_current
        #print(ts_current)
        ts_end = ts_current + pd.Timedelta(seconds=30)

        sum1 = 0
        sum2 = 0
        sum3 = 0
        rssi_df1 = 0
        rssi_df2 = 0
        rssi_df3 = 0

        rssi_curr_df1 = row['rssi']
        rssi_df1 = rssi_curr_df1
        #print(f'cur: {rssi_df1}')
        sum1 += 1
        #print(f'id: {index}')
        if index != 0:
            rssi_prev_df1 = df1_id.loc[index-1, 'rssi']
            #print(f'prev: {rssi_prev_df1}')
            rssi_df1 = rssi_df1 + rssi_prev_df1
            sum1 += 1 

        if index <= id_max-2:
            rssi_next_df1 =  df1_id.loc[index+1, 'rssi']
            #print(f'next: {rssi_next_df1}')
            rssi_df1 = rssi_df1 + rssi_next_df1
            sum1 += 1 
        
        rssi_df1 = rssi_df1/sum1
        df_rssi.loc[id, 'rssi1'] = rssi_df1
        #print(f'final 1: {rssi_df1}')

        # Find the corresponding next timestamp in df2 and df3
        df2_subset_next = df2_id[(df2_id['ts'] >= ts_current) & (df2_id['ts'] <= ts_end)]
        df3_subset_next = df3_id[(df3_id['ts'] >= ts_current) & (df3_id['ts'] <= ts_end)]

        if not df2_subset_next.empty:
            # Find the first occurrence of the next timestamp in df2 and df3
            ts_next_df2 = df2_subset_next.iloc[0]['ts']
            rssi_next_df2 = df2_subset_next.iloc[0]['rssi']
            #print(f'2 next: {ts_next_df2} rssi: {rssi_next_df2}')
            rssi_df2 = rssi_next_df2
            sum2 += 1
        if not df3_subset_next.empty:
            ts_next_df3 = df3_subset_next.iloc[0]['ts']
            rssi_next_df3 = df3_subset_next.iloc[0]['rssi']
            #print(f'3 next: {ts_next_df3} rssi: {rssi_next_df3}')
            rssi_df3 = rssi_next_df3
            sum3 += 1

        # Find the corresponding previous timestamp in df2 and df3
        df2_subset_prev = df2_id[(df2_id['ts'] <= ts_current)].tail(1)
        df3_subset_prev = df3_id[(df3_id['ts'] <= ts_current)].tail(1)

        # Find the first occurrence of the previous timestamp in df2 and df3
        if not df2_subset_prev.empty:
            ts_prev_df2 = df2_subset_prev.iloc[0]['ts']
            rssi_prev_df2 = df2_subset_prev.iloc[0]['rssi']
            #print(f'2 prev: {ts_prev_df2} rssi: {rssi_prev_df2}')
            rssi_df2 += rssi_prev_df2
            sum2 += 1

        if not df3_subset_prev.empty:
            ts_prev_df3 = df3_subset_prev.iloc[0]['ts']
            rssi_prev_df3 = df3_subset_prev.iloc[0]['rssi']
            #print(f'3 prev: {ts_prev_df3} rssi: {rssi_prev_df3}')
            rssi_df3 += rssi_prev_df3
            sum3 += 1
        
        if sum2 != 0:
            rssi_df2 = rssi_df2/sum2
            df_rssi.loc[id, 'rssi2'] = rssi_df2
        else: #CHECK 0 OR -100
            df_rssi.loc[id, 'rssi2'] = -100

        if sum3 != 0:
            rssi_df3 = rssi_df3/sum3
            df_rssi.loc[id, 'rssi3'] = rssi_df3
        else: #CHECK 0 OR -100
            df_rssi.loc[id, 'rssi3'] = -100

        #print()
        #print(f'1:{rssi_df1},2:{rssi_df2}, 3:{rssi_df3}' )
        #fazer rms com os valores de ref e ver qual da melhor accuracy
        #fazer funcao de rms menor, e comparar com o maior valor
        id += 1
    
    #display(df_rssi)
    return df_rssi

def calculate_rms(rssi, rssi_ref):
    # Calculate RMS difference for each receiver   
    rms_diff = np.sqrt(((rssi) - rssi_ref)**2)

    return rms_diff



df_rssi1 = analyse_df(df1,df2,df3, beacon = 1) #beacon 1

df_rssi2 = analyse_df(df1,df2,df3, beacon = 2) #beacon 1



#rssi_df1,rssi_df2, rssi_df3 = analyse_df(df1,df2,df3, beacon = 2) #beacon 2


In [6]:

def classify_point(df_rssi, ref_values):
    rms_station = np.zeros(3)

    for index, row in df_rssi.iterrows():

        #print(f"      {round(row['rssi1'],2)},      {round(row['rssi2'],2)},       {round(row['rssi3'],2)}")

        # the rms of beacon being in station 1
        rms_station[0] = (calculate_rms(row['rssi1'], ref_values[0,0]) + 
                        calculate_rms(row['rssi2'], ref_values[0,1]) +
                        calculate_rms(row['rssi3'], ref_values[0,2]))
        
        # the rms of beacon being in station 2
        rms_station[1] = (calculate_rms(row['rssi1'], ref_values[1,0]) + 
                        calculate_rms(row['rssi2'], ref_values[1,1]) +
                        calculate_rms(row['rssi3'], ref_values[1,2]))

        # the rms of beacon being in station 3
        rms_station[2] = (calculate_rms(row['rssi1'], ref_values[2,0]) + 
                        calculate_rms(row['rssi2'], ref_values[2,1]) +
                        calculate_rms(row['rssi3'], ref_values[2,2]))
        #print(f'rms 1: {round(rms_station[0],2)} rms 2: {round(rms_station[1],2)} rms 3: {round(rms_station[2],2)}')
        #print()
        #min_rms = min(rms_station)
        station_min = np.argmin(rms_station)
        station = station_min + 1

        df_rssi.loc[index, 'station'] = station
    return df_rssi

def time_in_station_ble(df_rssi):
    time_in_1 = pd.to_timedelta('0 seconds')
    time_in_2 = pd.to_timedelta('0 seconds')
    time_in_3 = pd.to_timedelta('0 seconds')
    for index, row in df_rssi.iterrows():
        if row['station'] == 1:
            if index == 0:
                continue
            time_in_1 += df_rssi.loc[index,'ts'] - df_rssi.loc[index-1,'ts']

        if row['station'] == 2:
            if index == 0:
                continue
            time_in_2 += df_rssi.loc[index,'ts'] - df_rssi.loc[index-1,'ts']

        if row['station'] == 3:
            if index == 0:
                continue
            time_in_3 += df_rssi.loc[index,'ts'] - df_rssi.loc[index-1,'ts']
    
    total_time = time_in_1+time_in_2+time_in_3
    perc1 = time_in_1/total_time * 100
    perc1 = round(perc1,2)
    perc2 = time_in_2/total_time* 100
    perc2 = round(perc2,2)
    perc3 = time_in_3/total_time* 100
    perc3 = round(perc3,2)
    print(time_in_1,time_in_2,time_in_3)
    print(time_in_1+time_in_2+time_in_3)
    print(f'{perc1}%,   {perc2}%,   {perc3}%')
    #display(df_rssi)

    return

df_rssi1 = classify_point(df_rssi1, ref_values)
time_in_station_ble(df_rssi1.loc[:50])
display(df_rssi1.loc[:50])
#display(df_rssi1)

df_rssi2 = classify_point(df_rssi2, ref_values)
time_in_station_ble(df_rssi2.loc[:50])
display(df_rssi2.loc[:50])
#display(df_rssi2)

0 days 00:00:18 0 days 00:00:00 0 days 00:12:25
0 days 00:12:43
2.36%,   0.0%,   97.64%


,ts,rssi1,rssi2,rssi3,station,x,y
0,2023-10-10 09:00:54,-62.5,-67.0,-62.0,3,NaN,NaN
1,2023-10-10 09:01:18,-61.0,-72.0,-62.0,3,NaN,NaN
2,2023-10-10 09:01:24,-64.333333,-72.0,-62.0,3,NaN,NaN
3,2023-10-10 09:02:00,-59.333333,-68.0,-61.5,3,NaN,NaN
4,2023-10-10 09:02:06,-59.333333,-68.0,-61.5,3,NaN,NaN
5,2023-10-10 09:02:30,-57.666667,-68.0,-68.0,3,NaN,NaN
6,2023-10-10 09:02:42,-65.666667,-77.0,-68.0,1,NaN,NaN
7,2023-10-10 09:03:24,-66.333333,-68.0,-64.5,3,NaN,NaN
8,2023-10-10 09:03:30,-68.0,-68.0,-67.5,3,NaN,NaN
9,2023-10-10 09:03:36,-64.0,-68.0,-67.5,3,NaN,NaN


0 days 00:03:30 0 days 00:02:12 0 days 00:07:02
0 days 00:12:44
27.49%,   17.28%,   55.24%


,ts,rssi1,rssi2,rssi3,station,x,y
0,2023-10-10 09:00:23,-66.5,-76.0,-66.5,1,NaN,NaN
1,2023-10-10 09:00:42,-68.0,-69.5,-66.5,3,NaN,NaN
2,2023-10-10 09:00:48,-65.333333,-69.5,-63.0,3,NaN,NaN
3,2023-10-10 09:00:54,-68.666667,-70.0,-67.5,3,NaN,NaN
4,2023-10-10 09:01:00,-66.0,-70.0,-67.5,3,NaN,NaN
5,2023-10-10 09:01:12,-69.0,-71.0,-67.5,3,NaN,NaN
6,2023-10-10 09:01:48,-70.333333,-78.0,-69.0,1,NaN,NaN
7,2023-10-10 09:01:54,-74.0,-78.0,-69.0,1,NaN,NaN
8,2023-10-10 09:02:18,-71.666667,-71.5,-70.0,3,NaN,NaN
9,2023-10-10 09:02:42,-72.0,-71.5,-70.5,3,NaN,NaN


In [8]:
import random
    
def plot_ble_state(df_rssi,beacon):
 
    x = np.array(df_rssi['ts'])
    y = np.array(df_rssi['station'])
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=x, y=y, name="hv",
                    line_shape='hv'))
    fig1.show()

def ble_spaghetti_plot(df_rssi):
    
    for index, row in df_rssi.iterrows():
        random_error = random.uniform(-0.1, 0.1)
        if row['station'] == 3:
            row['x'] = 0 + random_error
            row['y'] = 0 + random_error
        elif row['station'] == 2:
            row['x'] = 15 + random_error
            row['y'] = 4 + random_error
        elif row['station'] == 1:
            row['x'] = 0 + random_error
            row['y'] = 4 + random_error
    #display(df_rssi.loc[:50])

#df = pd.DataFrame(dict(
#    x = [15, 0, 0, 15],
#    y = [4, 4, 0, 4]
#))
    fig = px.line(df_rssi, x="x", y="y", title="Unsorted Input") 
    fig.show()

plot = 1

if plot:
    ble_spaghetti_plot(df_rssi1)
    plot_ble_state(df_rssi1,beacon = 1)
    ble_spaghetti_plot(df_rssi2)
    plot_ble_state(df_rssi2, beacon = 2)



   